In [1]:
## data processing ###
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torchvision.datasets as td
from torchvision import transforms
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix,f1_score
import torchvision
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torchvision.datasets as td
from torchvision import transforms

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive
!ls

/content/drive/My Drive
 Autoencoder.ipynb	  image_total		     Presentation.pdf
 DL_fp_deeplab-v3.ipynb  'label info.csv'	     sample_label.CSV
 DL_fp_TernausNet.ipynb   ML_fp_v1.ipynb	    'slide from dong.pptx'
 DL_fp_Unet.ipynb	  ML_fp_v3.ipynb	     test
 DL-HW3.ipynb		  ML_project_v2.ipynb	     train
 DL_HW4.ipynb		  model_paramenter.txt	     Untitled0.ipynb
 evaluate_image		  optimizer_paramenter.txt   Untitled1.ipynb


In [4]:
### load information table##
data=pd.read_csv('label info.csv',sep='\t')
data.head(15)

,Unnamed: 0,Name,format,creator,type,date,subject
0,0,0016285_RP-T-1900-A-4378.jpg,"['hoogte 187 MMmmmm', 'breedte 497 MMmmmm', 'm...","[' tekenaar: Cuyp, Aelbert']",['tekening'],['1630 - before 1649'],['Leiden']
1,1,0007673_BK-NM-12985.jpg,"['hoogte 15.6 CMcmcm', 'lengte 17.8 CMcmcm', '...","[' zilversmid: Vianen, Adam van (I)']","['coupe', 'drinkschaal']",['1621 - 1621'],"['Iconclasscode: 48A9862', 'Iconclasscode: 92L..."
2,2,0013705_RP-P-OB-1825.jpg,"['blad: hoogte 93 MMmmmm', 'blad: breedte 84 M...","['toegeschreven aan prentmaker: Leyden, Lucas ...","['prent', 'boekillustratie']",['1513 - 1517'],"['Pepijn, hertog van Brabant', 'Iconclasscode:..."
3,3,0048095_RP-P-1887-A-12285.jpg,"['plaatrand: hoogte 196 MMmmmm', 'plaatrand: b...","['vermeld op object prentmaker: Huys, Frans']",['prent'],['1546 - 1562'],['Iconclasscode: 11I2(PERSIAN)']
4,4,0057724_RP-P-OB-24.001.jpg,"['plaatrand: hoogte 75 MMmmmm', 'plaatrand: br...","['vermeld op object prentmaker: Nickelen, Jan ...",['prent'],['1665 - 1721'],"['Iconclasscode: 25H213', 'Iconclasscode: 25I6']"
5,5,0059662_RP-P-1906-1541.jpg,"['plaatrand: hoogte 171 MMmmmm', 'plaatrand: b...","['vermeld op object prentmaker: Pfeiffer, Fran...",['prent'],['1794 - 1794'],"['Willmet, Joannes']"
6,6,0107821_RP-P-OB-83.457.jpg,"['plaatrand: hoogte 120 MMmmmm', 'plaatrand: b...","['vermeld op object prentmaker: Audran, Jean',...",['prent'],['1680 - 1720'],['Iconclasscode: 92L4223']
7,7,0061854_RP-P-OB-7469.jpg,"['blad: hoogte 172 MMmmmm', 'blad: breedte 228...","['vermeld op object prentmaker: Sadeler, Johan...",['prent'],['1587 - 1587'],"['Iconclasscode: 21A', 'Iconclasscode: 92C1']"
8,8,0094478_RP-P-OB-78.623-32.jpg,"['plaatrand: hoogte 240 MMmmmm', 'plaatrand: b...","[' prentmaker: onbekend', ' naar prent van: Ho...","['prent', 'nieuwsprent']",['1573 - 1575'],"['Fort Rammekens', 'Iconclasscode: 45K4']"
9,9,0020635_RP-P-OB-5675.jpg,"['plaatrand: hoogte 163 MMmmmm', 'plaatrand: b...",['vermeld op object prentmaker: Gellée (genaam...,['prent'],['1635 - 1635'],"['Iconclasscode: 47I221', 'Iconclasscode: 49G3..."


In [ ]:
### the whole size of database###
type_list=np.asarray(data.type)
#u_name=np.unique(type_l)
len(type_list)


112039

In [ ]:
### choose 5 interesting label and test its size###
type1=data[data.type=='[\'bord (vaatwerk)\']'].shape[0] ## pottery##
type2=data[data.type=='[\'prent\']'].shape[0] ##print###
type3=data[data.type=='[\'tekening\']'].shape[0] ## painting##
type4=data[data.type=='[\'foto\']'].shape[0]  ## photo##
type5=data[data.type=='[\'beeldhouwwerk\']'].shape[0] ##sculpture##

print(f'{type1},{type2},{type3},{type4},{type5}')

519,52166,10882,1768,538


In [ ]:
###create train dataset###
list1=data[data.type=='[\'bord (vaatwerk)\']'].Name.values[:200] ##label 0
list2=data[data.type=='[\'prent\']'].Name.values[:200]  ##label 1
list3=data[data.type=='[\'tekening\']'].Name.values[:200] ##label 2
list4=data[data.type=='[\'foto\']'].Name.values[:200] ## label 3
list5=data[data.type=='[\'beeldhouwwerk\']'].Name.values[:200] ##label 4

sample_list=np.c_[list1,list2,list3,list4,list5]
sample_dataframe=pd.DataFrame(sample_list)
sample_dataframe.columns=[i for i in range(5)]




str

In [ ]:
## use this set as evaluation set###
test1=data[data.type=='[\'bord (vaatwerk)\']'].Name.values[201:204] 
test2=data[data.type=='[\'prent\']'].Name.values[201:204] 
test3=data[data.type=='[\'tekening\']'].Name.values[201:204]
test4=data[data.type=='[\'foto\']'].Name.values[201:204]
test5=data[data.type=='[\'beeldhouwwerk\']'].Name.values[201:204]
test_list=np.c_[test1,test2,test3,test4,test5]
test_dataframe=pd.DataFrame(test_list)
test_dataframe.columns=[i for i in range(5)]
test_dataframe

,0,1,2,3,4
0,0066684_BK-NM-4247.jpg,0060770_RP-P-1951-861.jpg,0023012_RP-T-1888-A-1614.jpg,0111892_NG-2011-29-21.jpg,0081647_BK-NM-144-8.jpg
1,0007174_BK-NM-12400-74.jpg,0050295_RP-P-1907-2362.jpg,0025832_RP-T-1984-12.jpg,0071248_RP-F-F25214-AB.jpg,0008906_BK-18813.jpg
2,0109971_BK-2010-15-7.jpg,0103409_RP-P-OB-82.623.jpg,0064760_RP-T-1933-86.jpg,0065920_BI-F-1921-317-63.jpg,0009102_BK-18771-B.jpg


In [5]:
### construct new test set###
###random choose 15 images per label###
np.random.seed(675)
##first label##
index_li1=np.where(data[data.type=='[\'bord (vaatwerk)\']'].Name.values)
label_f=data.Name.values[index_li1]
index_f=np.random.randint(0,519,15)
name_f=data[data.type=='[\'bord (vaatwerk)\']'].Name.values[index_f]
##second_label##
index_li2=np.where(data[data.type=='[\'prent\']'].Name.values)
label_s=data.Name.values[index_li2]
index_s=np.random.randint(0,52166,15)
name_s=data[data.type=='[\'prent\']'].Name.values[index_s]
### third_label##
index_li3=np.where(data[data.type=='[\'tekening\']'].Name.values)
label_t=data.Name.values[index_li3]
index_t=np.random.randint(0,10882,15)
name_t=data[data.type=='[\'tekening\']'].Name.values[index_t]
## fourth_label##
index_li4=np.where(data[data.type=='[\'foto\']'].Name.values)
label_fo=data.Name.values[index_li4]
index_fo=np.random.randint(0,1768,15)
name_fo=data[data.type=='[\'foto\']'].Name.values[index_fo]
###fif_label##
index_li5=np.where(data[data.type=='[\'beeldhouwwerk\']'].Name.values)
label_fif=data.Name.values[index_li5]
index_fif=np.random.randint(0,538,15)
name_fif=data[data.type=='[\'beeldhouwwerk\']'].Name.values[index_fif]

##construct combined name dataframe##
new_test=np.c_[name_f,name_s,name_t,name_fo,name_fif]
new_test_df=pd.DataFrame(new_test)
new_test_df.columns=[i for i in range(5)]
new_test_df.head(15)

,0,1,2,3,4
0,0091341_BK-KOG-2394.jpg,0060912_RP-P-OB-24.628.jpg,0024981_RP-T-1902-A-4613.jpg,0081795_RP-F-F80374.jpg,0008618_BK-NM-7206.jpg
1,0001499_AK-NM-13428.jpg,0109556_RP-P-OB-102.476.jpg,0065316_RP-T-1928-74.jpg,0070454_RP-F-F14392-B.jpg,0008920_BK-18811.jpg
2,0001166_AK-RBK-15858.jpg,0104535_RP-P-OB-83.034-203.jpg,0024987_RP-T-1961-95.jpg,0023489_RP-F-00-57.jpg,0081642_BK-NM-144-3.jpg
3,0066747_BK-NM-11168-B.jpg,0077929_RP-P-1885-A-9960.jpg,0065267_RP-T-00-1202.jpg,0092308_RP-F-2005-107-247.jpg,0008772_BK-1959-50-B.jpg
4,0091341_BK-KOG-2394.jpg,0059406_RP-P-1982-260.jpg,0022817_RP-T-1884-A-330E.jpg,0081892_RP-F-1997-65.jpg,0008939_BK-15401.jpg
5,0007103_BK-NM-1578-D.jpg,0026422_RP-P-OB-50.288.jpg,0054367_RP-T-1921-407.jpg,0065900_BI-F-1921-317-46.jpg,0009135_BK-NM-11961-A.jpg
6,0066837_BK-1975-43.jpg,0103861_RP-P-OB-76.644.jpg,0055099_NG-1985-7-1-113.jpg,0082026_RP-F-1997-93.jpg,0008774_BK-1961-49-B.jpg
7,0006666_BK-NM-12400-157.jpg,0088815_RP-P-1944-471.jpg,0010107_RP-T-1890-A-2247.jpg,0082534_RP-F-AB12279-AA.jpg,0075875_BK-C-1994-1.jpg
8,0006580_BK-NM-3898.jpg,0080929_RP-P-1983-15.jpg,0026204_RP-T-1931-174.jpg,0023417_RP-F-1928-31.jpg,0008954_BK-15399.jpg
9,0001462_AK-NM-13395.jpg,0044014_RP-P-OB-55.901.jpg,0025336_RP-T-1910-31.jpg,0070390_RP-F-F00622.jpg,0008807_BK-NM-3326.jpg


In [ ]:
result=pd.read_csv('sample_label.CSV',sep='\,')
type(result.iloc[0,0])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


str